<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Initialization-cell---run-this-first" data-toc-modified-id="Initialization-cell---run-this-first-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialization cell - run this first</a></span></li><li><span><a href="#Problem-formulation:-Finding-synchronized-waveforms-in-a-dataset" data-toc-modified-id="Problem-formulation:-Finding-synchronized-waveforms-in-a-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Problem formulation: Finding synchronized waveforms in a dataset</a></span><ul class="toc-item"><li><span><a href="#An-SVD-based-algorithm-for-finding-synced-waveforms" data-toc-modified-id="An-SVD-based-algorithm-for-finding-synced-waveforms-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>An SVD based algorithm for finding synced waveforms</a></span></li><li><span><a href="#Examining-the-noise-sensitivity-of-the-algorithm" data-toc-modified-id="Examining-the-noise-sensitivity-of-the-algorithm-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Examining the noise sensitivity of the algorithm</a></span></li><li><span><a href="#Testing-algorithm-in-setting-when-there-is--&quot;bias&quot;-in-unsynchronized-sensors" data-toc-modified-id="Testing-algorithm-in-setting-when-there-is--&quot;bias&quot;-in-unsynchronized-sensors-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Testing algorithm in setting when there is  "bias" in unsynchronized sensors</a></span></li><li><span><a href="#Extension:-Robustifying-algorithm-to-&quot;bias&quot;-in-unsynchronized-sensors" data-toc-modified-id="Extension:-Robustifying-algorithm-to-&quot;bias&quot;-in-unsynchronized-sensors-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Extension: Robustifying algorithm to "bias" in unsynchronized sensors</a></span></li></ul></li><li><span><a href="#A-robust-to-bias-eigen-algorithm-for-finding-synchronized-waveforms" data-toc-modified-id="A-robust-to-bias-eigen-algorithm-for-finding-synchronized-waveforms-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>A robust-to-bias eigen-algorithm for finding synchronized waveforms</a></span></li><li><span><a href="#Testing-algorithm-in-setting-with--&quot;approximate&quot;-synchronization" data-toc-modified-id="Testing-algorithm-in-setting-with--&quot;approximate&quot;-synchronization-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Testing algorithm in setting with  "approximate" synchronization</a></span></li><li><span><a href="#A-robust-to-bias-algorithm-to-find-approximately-synced-waveforms" data-toc-modified-id="A-robust-to-bias-algorithm-to-find-approximately-synced-waveforms-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>A robust-to-bias algorithm to find approximately synced waveforms</a></span><ul class="toc-item"><li><span><a href="#Signal-Processing-101:-Lagged-cross-correlation-of-two-vectors" data-toc-modified-id="Signal-Processing-101:-Lagged-cross-correlation-of-two-vectors-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Signal Processing 101: Lagged cross-correlation of two vectors</a></span></li><li><span><a href="#Challenge:-Find-synced-waveforms-in-a-larger-dataset" data-toc-modified-id="Challenge:-Find-synced-waveforms-in-a-larger-dataset-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Challenge: Find synced waveforms in a larger dataset</a></span></li></ul></li><li><span><a href="#A-conceptual-challenge-problem" data-toc-modified-id="A-conceptual-challenge-problem-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>A conceptual challenge problem</a></span><ul class="toc-item"><li><span><a href="#Optional-challenge-problem" data-toc-modified-id="Optional-challenge-problem-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Optional challenge problem</a></span></li></ul></li><li><span><a href="#A-substantially-faster-algorithm-using-the-FFT" data-toc-modified-id="A-substantially-faster-algorithm-using-the-FFT-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>A substantially faster algorithm using the FFT</a></span><ul class="toc-item"><li><span><a href="#Signal-Processing-201:-Discrete-Fourier-transform-of-circularly-shifted-waveforms" data-toc-modified-id="Signal-Processing-201:-Discrete-Fourier-transform-of-circularly-shifted-waveforms-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Signal Processing 201: Discrete Fourier transform of circularly shifted waveforms</a></span></li><li><span><a href="#Challenge-problem" data-toc-modified-id="Challenge-problem-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Challenge problem</a></span></li></ul></li></ul></div>

# Initialization cell - run this first

In [ ]:
using Plots, Interact, Colors, ProgressMeter, Arpack, LinearAlgebra, Statistics
gr(
    markerstrokewidth=0.3,
    markerstrokecolor="white",
    label="",
    markersize=4,
)
include("deps.jl")

# Problem formulation: Finding synchronized waveforms in a dataset 

We are interested in finding $k$ synchronized waveforms in a noisy $m \times n$ data matrix. We begin with a representative example of such a  waveform 

$$x(t;~a,~b) = \exp(-a\,t) \cdot \sin(b\,t),$$

where $a$ and $b$ are parameters The function `generate_spikewaveform` returns `n` samples of this waveform for different choices of `a` and `b`. 

In [ ]:
function generate_spikewaveform(n::Integer=1000, a::Number=50.0, b::Number=100.0)
    t = (0.0:1.0:(n - 1.0)) / n
    return exp.(-a * t) .* (sin.(b * t)), t
end

In [ ]:
@manipulate for a in (20, 50, 100), b in (50, 100, 150)
    n = 1000
    x, t = generate_spikewaveform(n, a, b)
    plot(t, x; xlabel="t", ylabel="x(t)")
end

This **pulse-like** characteristic of the waveform is imporant and a defining attribute of many interesting signals such as [ECG waveforms](https://en.wikipedia.org/wiki/Electrocardiography) and [neuronal action potentials](https://en.wikipedia.org/wiki/Action_potential).

For the remainder of this codex, we will work with the  (normalized) waveform as in the following cell 

In [ ]:
n = 1000
x, t = generate_spikewaveform(n, 50, 100)
x = x / norm(x)
plot(t, x; xlabel="t", ylabel="x(t)")

 We are interested in the problem of finding **synchronized waveforms** in an $m \times n$ data matrix. In this setup we think of $m$ as being the number of sensors and $n$ as being the number of samples or measurements; thus $X[i,:]$ represents the vector of time series measurements or samples recorded at sensor $i$.  
 
In the simplest model of our problem, we assume that if $i_{\sf sync}$ is a $k$-dimensional list of indices corresponding to the sensors that observe the synchronized waveform, then: 

$$ X[i,:] = \begin{cases} x^T & \textrm{ for } i \in i_{\sf sync}\\
                          0 & \textrm{ otherwise}. \end{cases}$$
                          
The subsequent code cell generates such a dataset and displays it as a [ridgeline](https://www.data-to-viz.com/graph/ridgeline.html)  (or [joy plot](https://serialmentor.com/blog/2017/9/15/goodbye-joyplots)). 

In [ ]:
m, k = 20, 5
sync_idx = [3, 7, 13, 18, 20]
X = zeros(m, n)
for idx in sync_idx
    X[idx, :] = x'
end
labels = ["sensor $i" for i in 1:m]
joyplot(permutedims(X); labels=labels, subplot_scale=1.0)

Indeed -- we verify this numerically next. 

In [ ]:
@show rank(X);

We verify this numerically next. 

In [ ]:
@show maximum(svdvals(X))
@show sqrt(k)
scatter(svdvals(X); xlabel="index", ylabel="singular value")

We **display** the elements of the **largest left singular vector** of $X$ next.  

In [ ]:
u₁ = svds(X; nsv=1)[1].U[:]
@show maximum(abs.(u₁))
@show maximum(u₁)
@show 1 / sqrt(k)
scatter(u₁; xlabel="i", ylabel="U[i, 1]")

In [ ]:
u₁ = sign.(u₁[findmax(abs.(u₁))[2]]) * u₁
@show extrema(u₁)
@show sortperm(u₁; rev=true)
@show sync_idx
scatter(u₁; xlabel="i", ylabel="U[i, 1]")

## An SVD based algorithm for finding synced waveforms

This motivates the use of $u_1$ for determining the indices (or sensor ids) that have synchronized waveforms.  Complete the `synced_vector_svd` function below -- it takes as its input the data matrix and returns as its output $u_1$.

In [ ]:
using Arpack

function synced_vector_svd(A::Matrix)
    U = svds(A; nsv=1)[1].U
    U = U * sign.(U[findmax(abs.(U))[2]]) # to fix sign
    return U
end

## Examining the noise sensitivity of the algorithm

What happens if we add noise? 

In [ ]:
@manipulate for σ in (0, 0.005, 0.05, 0.1, 0.5)
    X_noisy = X + σ * randn(size(X))
    u₁ = synced_vector_svd(X_noisy)[:]
    top5_coord_idx = sort(sortperm(u₁; rev=true)[1:5])
    synced_vector_svd(X_noisy)
    plot(
        joyplot(permutedims(X_noisy); labels=labels),
        scatter(
            u₁; 
            title="top 5 coords. idx = $top5_coord_idx", 
            xlabel="i", 
            ylabel="U{i, 1}",
            ylim=(-0.1, 1)
            ),
        layout=(1, 2),
        size=(900, 400)
    )
end

Having established that the algorithm *appears* to work and is robust to Gaussian noise, let us consider some real-world scenarios where the data collected differs from the ideal synchronized model we assumed. Our goal is to observe whether the algorithm still "works", and if/when it fails, to modify the algorithm so that it works again. 

## Testing algorithm in setting when there is  "bias" in unsynchronized sensors

Recall that the data model that we had earlier assumed that

$$ X[i,:] = \begin{cases} x^T & \textrm{ for } i \in i_{\sf sync}\\
                          0 & \textrm{ otherwise}. \end{cases}$$
                          
The assumption that $X[i,:] = 0$ for $i \notin i_{\sf sync}$ is a pretty strong assumption for real-world applications beacuse there might be a DC offset (or bias) even when there are no signals in a sensor. So, we instead consider data having the model

$$ Z[i,:] = \begin{cases} x^T & \textrm{ for } i \in i_{\sf sync}\\
                          c_i & \textrm{ otherwise}. \end{cases},$$

where $c_i$ is a constant that depends only on the sensor index $i$ but *not* on time. 
The subsequent code cell generates such a dataset 

In [ ]:
m, k = 20, 5
sync_idx = [3, 7, 13, 18, 20]
c = 0.1
Z = zeros(m, n)
for idx in 1:m  
    if idx in sync_idx 
        Z[idx, :] = x' 
    else
        Z[idx, :] = c * ones(n)'
    end
end
labels = ["sensor $i" for i in 1:m]
joyplot(permutedims(Z); labels=labels)

In [ ]:
u₁ = synced_vector_svd(Z)[:]
top5_coord_idx = sort(sortperm(u₁; rev=true)[1:5])
scatter(
    u₁;
    title="top-5 coords. idx = $top5_coord_idx", 
    xlabel="i", 
    ylabel="U{i,1}", 
    ylim=(-0.1, 1)
)

We now investigate this more systematically for different values of $c$. 

In [ ]:
@manipulate for c in (0, 0.1, 0.5), σ in (0, 0.05, 0.1, 0.5)
    Z = zeros(m, n)
    for idx in 1:m  
        if idx in sync_idx 
            Z[idx, :] = x' 
        else
            Z[idx, :] = c * ones(n)'
        end
    end
    Znoisy = Z + σ * randn(size(Z))
    u₁ = synced_vector_svd(Znoisy)[:]
    top5_coord_idx = sort(sortperm(u₁; rev=true)[1:5])
    plot(
        joyplot(permutedims(Znoisy); labels=labels),
        scatter(
            u₁; 
            title="top-5 coords. idx = $top5_coord_idx", 
            xlabel="i", 
            ylabel="U[i, 1]",
            ylim=(-0.1, 1)
            ),
        layout=(1, 2),
        size=(800, 400)
    )
    end

## Extension: Robustifying algorithm to "bias" in unsynchronized sensors

Recall the model with bias in the unsynchronized sensors.

$$ Z[i,:] = \begin{cases} x^T & \textrm{ for } i \in i_{\sf sync}\\
                          c_i & \textrm{ otherwise}. \end{cases},$$

where $c_i$ is a constant that depends only on the sensor index $i$ but *not* on time.

Indeed! The rank of $Z$ as verified in the next cell.`

In [ ]:
@show rank(Z)
scatter(svdvals(Z); xlabel="index", ylabel="singular value")

Our  **insight** that

$$Z = ex^T + c1^T,$$

reveals that the $u_1(Z)$ will be a linear combination of $e$ and $c$, as will $u_2(Z)$ -- they must because they are in the $\mathcal{R}(Z)$.

Consequently, the algorithm that utilizes $u_1(Z)$ to find the syncrhonized waveforms **fails** when $c \neq 0$.

**Exercise**:

Motivated by this insight, complete the function `synced_vector_robust2mean` so that it computes the leading left singular vector of the data matrix after the mean has been subtracted. 

In [ ]:
using Statistics: mean
using Arpack: svds

function synced_vector_robust2mean(A::Matrix)
    c = mean(A, dims=2)
    A = A .- c
    U = svds(A; nsv=1)[1].U
    U = U * sign.(U[findmax(abs.(U))[2]]) # to fix sign
    return U
end

We now test the algorithm for $ c\neq 0$ and various values of $\sigma$.

In [ ]:
@manipulate for c in (0, 0.05, 0.1, 0.5), σ in (0, 0.05, 0.1, 0.5)
    Z = zeros(m, n)
    for idx in 1:m  
        if idx in sync_idx 
            Z[idx, :] = x' 
        else
            Z[idx,:] = c * ones(n)'
        end
    end
    Znoisy = Z + σ * randn(size(Z))
    u₁ =  synced_vector_robust2mean(Znoisy)[:]
    top5_coord_idx = sort(sortperm(u₁; rev=true)[1:5])
    plot(
        joyplot(permutedims(Znoisy); labels=labels),
        scatter(u₁; title = "top-5 coords. idx = $top5_coord_idx", 
            xlabel="i", 
            ylabel="U[i,1]",
            ylim=(-0.1, 1)),
        layout=(1, 2),
        size=(800, 400)
    )
    end

# A robust-to-bias eigen-algorithm for finding synchronized waveforms 


The algorithm we have developed does not utilize the right singular vectors. We can thus express it as an eigen-algorithm involving the leading eigenvector of a symmetrix matrix $K$ as described next. 

Define the $m \times m$ matrix kernel or **similarity matrix**:

$$ K_{ij} = \langle X[i,:] - c_i,  X[j,:] - c_j\rangle :=  \left(X[i,:] - c_i\right) \cdot \left(X[j,:] - c_j\right)^T ,$$

**Fact**: 

Let $q_1(K)$ denote the largest eigenvector of $K$ and $u_1(X)$ denote the largest left singular vector of $X$. Then it can be shown that : 

$$q_1(K) = u_1(X - c1^T).$$


**Exercise**: 

Utilize this fact to complete the `synced_vector` function which takes as its input the data matrix and returns as its output `K` and `u` where $u= q_1(K)$. 

In [ ]:
using LinearAlgebra: dot, Diagonal, diag
using Statistics: mean
using Arpack: eigs

function synced_vector(A::Matrix)
    m = size(A, 1)
    K = zeros(m, m)
    
    # Only compute lower triangle part
    for i in 1:m
        for j in 1:i
            K[i, j] = dot(A[i, :] .- mean(A[i, :]), A[j, :] .- mean(A[j, :]))
        end
    end

    # Complete the matrix
    K = K + K' - Diagonal(diag(K))
    U = eigs(K; nev=1, which=:LR)[2]
    U = U * sign(U[findmax(abs.(U))[2]]) # to fix sign
    return U, K
end

We now display the matrix $K$ as a heatmap. 

In [ ]:
Kx = synced_vector(X)[2]
@show rank(Kx)
heatmap(Kx; aspect_ratio=1.0, axis=false, yflip=:true, grid=false)

This **clique like** pattern of connections in the heatmap of $K$ is the signature for the synchronized waveforms.

We now examine if this signature persists, or how it changes when we add noise and bias to the data.

In [ ]:
@manipulate for c in (0, 0.05, 0.1, 0.5), σ in (0, 0.01, 0.05,0.5)
    Z = zeros(m, n)
    for idx in 1:m  
        if idx in sync_idx 
            Z[idx, :] = x' 
        else
            Z[idx, :] = c * ones(n)'
        end
    end
    Znoisy = Z + σ * randn(size(Z))
    u₁, K = synced_vector(Znoisy)
    u₁ = u₁[:]
    top5_coord_idx = sort(sortperm(u₁; rev=true)[1:5])
    p = plot(
        joyplot(permutedims(Znoisy); labels=labels),
        scatter(
            u₁; 
            title="max_idx = $top5_coord_idx", 
            xlabel="i", 
            ylabel="U[i, 1]",
            ylim=(-0.1, 1)
            ),
        heatmap(K; aspect_ratio=1.0),
    layout=(1, 3),
    size=(900, 350)
    )
end

The diagonal element corresponds to $||X[i,:]||_2^2$ corresponding to the unsyncrhonized indices where there is noise when $\sigma \neq 0$. As we have seen, the heatmap of $K$  will provide valuable information when the improved robust-to-bias algorithm does not succeed, as we shall see next.   

#  Testing algorithm in setting with  "approximate" synchronization

We now test the algorithm in a setting where the waveforms are **approximately synchronized**. The model we have in mind is that


$$ \tilde{X}[i,:]= \begin{cases} \textrm{shift}_{\delta_i}\left(x^T\right) + \textrm{noise} & \textrm{ for } i \in i_{\sf sync}\\
                          c_i + \textrm{noise} & \textrm{ otherwise}. \end{cases},$$

where,as before,  $c_i$ is a constant that depends only on the sensor index $i$ but *not* on time and $\delta_i$ is the shift (or drift) of the waveform in sensor $i$. The (circular) **shift operator** $\textrm{shift}^{\delta}(\cdot)$  shifts the entries of a vector rightwards by $\delta$ units. 

Mathematically, if $y$ is a vector with $n$ dimensions then

$$ \textrm{shift}_{\delta}\left(y\right) =  y[ i+\delta \textrm{ modulo } n],$$

so that when $\delta = 1$ and 

$$ y = \begin{bmatrix} y_1 & y_2 & \ldots & y_{n-1} & y_n\end{bmatrix},$$

then 

$$\textrm{shift}_{\delta}\left(y\right)  = \begin{bmatrix} y_2 & \ldots & y_{n-1} & y_1 \end{bmatrix},$$

and so on. 

The function `randshift` shifts the waveforms in this manner. 

In [ ]:
function randshift(X::Matrix, shift::Vector{Int}=rand(75:150, size(X, 1)))
    Xshift = zeros(size(X))
    for idx in 1:length(shift)
        Xshift[idx, :] = circshift(X[idx, :]', (0, shift[idx]))
    end
    return Xshift
end

What this code accomplishes is best illustrated graphically via a concrete example as in the next code cell. 

In [ ]:
shift_vector = zeros(Int64, m)
shift_vector[sync_idx] = [50; 160; 200; 320; 420]
Xshift = randshift(X, shift_vector)
labels =["sensor $i" for i in 1:m]
joyplot(permutedims(Xshift); labels=labels)

We would still like to find these approximately synchronized waveforms. We now test our previous algorithm to see if it still succeeds.

In [ ]:
@manipulate for c in (0, 0.05, 0.1, 0.5), σ in (0, 0.01, 0.05,0.5)
    Z = zeros(m, n)
    for idx in 1:m  
        if idx in sync_idx 
            Z[idx, :] = x' 
        else
            Z[idx, :] = c * ones(n)'
        end
    end
    Znoisy = Z + σ * randn(size(Z))
    shift_vector[sync_idx] = [50; 160; 200; 320; 420]
    Zshift_noisy = randshift(Znoisy, shift_vector)
    u₁, K = synced_vector(Zshift_noisy)
    u₁ = u₁[:]
    top5_coord_idx = sort(sortperm(u₁; rev=true)[1:5])
    p = plot(
        joyplot(permutedims(Zshift_noisy); labels=labels),
        scatter(
            u₁;
            title="max_idx = $top5_coord_idx", 
            xlabel="i", 
            ylabel="U[i, 1]",
            ylim=(-0.1, 1)),
        heatmap(K; aspect_ratio=1.0), layout=(1, 3),
        size=(900, 400)
    )
end

**Exercise**:


-- What structure for $K$ emerges instead of the clique-like structure?

-- What is the rank of $K$ we expect versus what we get?

-- Can you explain why this leads to the failure of the eigen-algorithm?

**Write down your answers and discuss it with the instructors when ready. **

In [ ]:
## TODO: Your code for examining rank, eigenvectors and structure of K 
c = 0.0
σ = 0.0
Z = zeros(m, n)
for idx in 1:m
     if idx in sync_idx
         Z[idx, :] = x'
     else
         Z[idx, :] = c * ones(n)'
     end
end
Znoisy = Z + σ * randn(size(Z))
shift_vector[sync_idx] = [50; 160; 200; 320; 420]
Zshift_noisy = randshift(Znoisy, shift_vector)
u₁, K = synced_vector(Zshift_noisy)

@show rank(K)
plot(
     joyplot(permutedims(Zshift_noisy); labels=labels),
     heatmap(K; aspect_ratio=1.0, axis=false, yflip=:true, grid=false),
     layout = (1,2))


# A robust-to-bias algorithm to find approximately synced waveforms

The  key insight is that we have to change how the $K$ matrix is computed so that we **induce rank one structure**  even when the synced waveforms are shifted. To that end we introduce the notion of cross-correlation of vectors that will come in handy.

## Signal Processing 101: Lagged cross-correlation of two vectors

Consider the two signals as in the next cell. 

In [ ]:
x = randn(50)
y = circshift(x, 10)
joyplot(hcat(x, y))

These signals are related by a circular shift. 

The inner-product of these two vectors given by

$$ \langle x, \rangle y = x^T y = \sum x_i y_i$$

We compute these numerically for `x` and `y` in the next code cell. 

In [ ]:
@show dot(x, y);

**Cross-correlation**:

Let $\tau$ be a lag (or delay). Then the lag $\tau$ cross-correlation between $x$ and y$ is defined as:

$$ \rho_{xy}(\tau) = \langle x, \textrm{shift}_{-\tau}(y) \rangle.$$

In other words, we shift $y$ *leftwards* by $\tau$ units and compute the usual inner product.

The function `cross_correlate` in the next cell does preceisly this for a range of values for $\tau$.  

In [ ]:
function cross_correlate(x, y, maxlag=length(x))
    cross_corr = []
    lags = 0:maxlag - 1
    for lag in 0:maxlag-1
         cross_corr_lag = dot(x, circshift(y, -lag))
         cross_corr = push!(cross_corr, cross_corr_lag)
    end    
    return cross_corr, collect(lags)
end

We now illustrate the utility of the cross-correlation function by computing by the cross-correlation between `x` and `y` for a range of lag values and displaying some pertinent statistics, as in the next cell.  

In [ ]:
cross_corr_xy, lags = cross_correlate(x, y)
@show x' * x
@show best_lag_idx = findmax(abs.(cross_corr_xy))[2]
@show best_lag = lags[best_lag_idx]
@show cross_corr_xy[best_lag_idx]
scatter(lags, cross_corr_xy; xlabel="lag", ylabel="cross-correlation")

We now examine what the cross correlation function looks like when the variables being cross-correlated are random, unlike in the previous setting where they were simply delayed versions of each other. 

In [ ]:
x = randn(50)
z = randn(50)
joyplot(hcat(x, z))

In [ ]:
cross_corr_xz, lags = cross_correlate(x,z)
@show maximum(abs.(cross_corr_xz))
@show best_lag_idx = findmax(abs.(cross_corr_xz))[2]
@show best_lag = lags[best_lag_idx]
@show cross_corr_xz[best_lag_idx]
scatter(lags, cross_corr_xz)



**Exercise**: 

Complete the `robust2shift_synced_vector` by modifying the `K` matrix definition so that it induces a rank-one matrix for shifted synced waveforms.
    
Hint: You will need to use the `cross_correlate` function and utilize your insights on how the cross-correlation function differens when signals are synced but shifted versus unrelated/randomly related. 
        



In [ ]:
function robust2shift_synced_vector(A::Matrix, maxlag=size(A, 2))
    m, n = size(A)
    K = zeros(m, m)
    @showprogress for i in 1:m
        for j in 1:i
            
            ##TODO: You might need more than one line  -- you'll have to repeat the computation sketched out above 
            cross_corr_ij, lags = cross_correlate(A[i, :], A[j, :])
            best_lag_idx = findmax(abs.(cross_corr_ij))[2]
            K[i, j] = cross_corr_ij[best_lag_idx] 
        end
    end
    
    # Complete the matrix
    K = K + K' - Diagonal(diag(K))
    ## Keep unchanged 
    U = eigs(K; nev=1, which=:LR)[2]
    U = U * sign.(U[findmax(abs.(U))[2]]) # to find sign
    return U, K 
end

**Exercise**:

Test the algorithm on `Xshift` to verify if the algorithm does indeed work as designed.

In [ ]:
##TODO: Test algorithm on Xshift`

u₁, K = robust2shift_synced_vector(Xshift)
# Plot disapleyd vector

u₁ = u₁[:]
top5_coord_idx = sort(sortperm(u₁; rev=true)[1:5])
p = plot(
    joyplot(permutedims(Xshift); labels=labels),
    scatter(
            u₁;
            title="max_idx = $top5_coord_idx", 
            xlabel="i", 
            ylabel="U[i, 1]",
            ylim=(-0.1, 1)),
    heatmap(K; aspect_ratio=1.0), layout=(1, 3),
    size=(900, 400)
    
)

**Exercise**: What about the plot reveals that the algorithm works as intended.

Acknowledge that when the algorithm succeeded again it is because the  𝐾  matrix had a clique-like structure and was rank 1.

**Exercise**: 

We now  test it on `Xshift + noise + bias` to see if it still returns the correct values.

In [ ]:
##TODO: Test algorithm on Xshift + noise + bias for different values 
n = 1000
x, t = generate_spikewaveform(n, 50, 100)
x = x / norm(x)
## Plot displeyd vector
@manipulate for c in (0, 0.05, 0.1, 0.5), σ in (0, 0.01, 0.05,0.5)
    Z = zeros(m, n)
    for idx in 1:m  
        if idx in sync_idx 
            Z[idx, :] = x' 
        else
            Z[idx, :] = c * ones(n)'
        end
    end
    Znoisy = Z + σ * randn(size(Z))
    shift_vector[sync_idx] = [50; 160; 200; 320; 420]
    Zshift_noisy = randshift(Znoisy, shift_vector)
    u₁, K = robust2shift_synced_vector(Zshift_noisy)
    u₁ = u₁[:]
    top5_coord_idx = sort(sortperm(u₁; rev=true)[1:5])
    p = plot(
        joyplot(permutedims(Zshift_noisy); labels=labels),
        scatter(
            u₁;
            title="max_idx = $top5_coord_idx", 
            xlabel="i", 
            ylabel="U[i, 1]",
            ylim=(-0.1, 1)),
        heatmap(K; aspect_ratio=1.0), layout=(1, 3),
        size=(900, 400)
    )
end

**Exercise**: What about the plot reveals that the algorithm works as intended.

For different values of c and σ, the clique-like patterns in the heatmap of Kx persists for σ = 0 and σ = 0.01 and the different values of c and for σ = 0.05 and the different values of c there is also a strong diagonal component. Except when σ = 0.5 when the checkerboard pattern vanishes and so does the ability of the algorithm to find the synchronized waveforms due to noisy signal mask the synchronized waveforms. So in conclusion we can say  the algorithm succeeded again it is because the 𝐾 matrix had a clique-like structure and was rank 1.

If it does, we are ready to test it on a larger dataset.

## Challenge: Find synced waveforms in a larger dataset

We first load the dataset.

In [ ]:
using JLD
Ydata = load("challenge_data.jld")["Yshift_noisy"]

Then compute the leading eigenvector using the algorithm we have designed.

In [ ]:
@time u₁, K = robust2shift_synced_vector(Ydata)
u₁ = u₁[:]

**Exercise**:

Determine the synched coordinates (we know that five of them are synchronized) in the next cell.  

In [ ]:
@show top5_coord_idx = sort(sortperm(u₁; rev=true)[1:5]) ## Enter code from above 
@time plot(
    scatter(
            u₁;
            title="max_idx = $top5_coord_idx", 
            xlabel="i", 
            ylabel="U{i, 1}",
            ylim=(-0.1,1)
        ),
    heatmap(
        K; 
        aspect_ratio=1.0
        );
    layout=(1, 2),
    size=(900, 400)
)

**Exercise**:

Fill in the indices corresponding to the synced waveformsbased on the output of your `robust2shift_synced_vector` code.

Hint: There are five such waveforms. 

In [ ]:
function nsynced()
    vector_idx  = [11, 49, 88, 101, 190] ##TODO: Five integer values 
    return sort(vector_idx)
end



![nsync](2o9p47.jpg)

To show that you did indeed find the (n)synced signals here is a joy(ous) plot of the waveforms.

In [ ]:
joyplot(Array(Ydata[nsynced(), :]'))

# A conceptual challenge problem

Suppose we are in a setting where the waveforms are still **approximately synchronized** but that the model we have in mind is that


$$ \tilde{X}[i,:]= \begin{cases} \alpha_i \textrm{shift}_{\delta_i}\left(x^T\right) + \textrm{noise} & \textrm{ for } i \in i_{\sf sync}\\
                          c_i + \textrm{noise} & \textrm{ otherwise}. \end{cases},$$

where $\alpha_i \in \mathbb{R}$ is a scalar and, as before,  $c_i$ is a constant that depends only on the sensor index $i$ but *not* on time and $\delta_i$ is the shift (or drift) of the waveform in sensor $i$. The (circular) **shift operator** $\textrm{shift}^{\delta}(\cdot)$  shifts the entries of a vector rightwards by $\delta$ units. 

The **main difference** in this model compared to the previous model is the presence of the $\alpha_i$ term. When $\alpha_i = 1$ we revert to the setup we have just solved. When $\alpha_i \neq 1$ then we are in a situation where the waveforms are still approximately synchronized but where the amplitudes are not the same.


**Exercise**:

For this setup: 

-- What is the rank of the `K` matrix returned by the  `robust2shift_synced_vector` from earlier?

-- What is the structure of the leading eigenvector? Hint: It should depend on the $\alpha_i$'s.

-- If we use the  same decision rule then mathematically express the rule for determining the synchronized indices

-- How does the output of this decison rule depend on $\alpha_i$?  

-- What does the dependence of the decision rule on $\alpha_i$ imply in practice? Hint: Do we know the $\alpha_i$'s?  What about this dependence is undesirable?




In [ ]:
n = 1000
x, t = generate_spikewaveform(n, 50, 100)
x = x / norm(x)
Z = zeros(m, n)
c = 0.05
σ = 0.05
for idx in 1:m  
    if idx in sync_idx 
        Z[idx, :] = rand() .* x' 
    else
        Z[idx, :] = c * ones(n)'
    end
end
Znoisy = Z + σ * randn(size(Z))
shift_vector[sync_idx] = [50; 160; 200; 320; 420]
Zshift_noisy = randshift(Znoisy, shift_vector)
u₁, K = robust2shift_synced_vector(Zshift_noisy)
@show rank(K)
m
#u₁ = u₁[:]
#top5_coord_idx = sort(sortperm(u₁; rev=true)[1:5]

the rank of the K matrix returned by the robust2shift_synced_vector from earlier is m(the number of sensors) if it contains noise, otherwise, the rank of K is 1.
$$$$
The structure of the leading eigenvector should satisfies $q_1 = \sum_i e_i \alpha_i for i \in i_{sync}$ and $rank(Q) = 1 $, in the noise-free setting.
$$$$
If $\alpha_i$ is large, it will outputs index corresponding to $\alpha_i$, if $\alpha_i$ is small, it will outputs index randomly.
$$$$
if $\alpha_i$ are relatively large, it will be easier to find synced waveforms from a noisy environment. On the
contrary, if $\alpha_i$ are small, the signal is less robust to noises and hard to tell apart from them

**Exercise**:

Describe a construction of the `K` matrix, different from that used in the `robust2shift_synced_vector` function that, in the noiseless setting eliminates the dependence on $\alpha_i$.

Hint: The computation will involve an appropriate normalization such that even when $\alpha_i$ is large and positive or large and negative we get $K_{ii} = 1$. 

$K[i,j]=  max(abs(cross\_correlate(norm(A[i, :]-mean(A[i,:]), norm(A[j,:]-mean(A[j,:]))))$

**Exercise**:

Complete the function `robust2shiftscale_synced_vector`. 

In [ ]:
function robust2shiftscale_synced_vector(A::Matrix, maxlag=size(A, 2))
    m, n = size(A)
    K = zeros(m, m)
    c = mean(A; dims = 2)
    A  = A .- c
    @showprogress for i in 1:m
        
        if norm(A[i,:]) > 1e-9
            A[i, :] /= norm(A[i, :])
        end
        for j in 1:i 
            cross_corr_ij, lags = cross_correlate(A[i, :], A[j, :])
            best_lag_idx = findmax(abs.(cross_corr_ij))[2]
            K[i, j] = cross_corr_ij[best_lag_idx] 
        end
    end
    
    # Complete the matrix
    K = K + K' - Diagonal(diag(K))
    ## Keep unchanged 
    U = eigs(K; nev=1, which=:LR)[2]
    U = U * sign.(U[findmax(abs.(U))[2]]) # to find sign
    return U, K 
end

## Optional challenge problem

Illustrate the improvement in performance due to the new algorithm that accounts for the scaling ambiguity relative to the algorithm that does not take into account this scaling ambiguity.

In [ ]:
##TODO: Your code here comparing robust2shiftscale_synced_vector and robust2shif_synced_vector 
##      illustrating superiority of the former
α_list = [3,6,9,12,15]
@manipulate for c in (0, 0.05, 0.1, 0.5), σ in (0, 0.01, 0.05,0.5)
     Z = zeros(m, n)
     i=1
    for idx in 1:m
         if idx in sync_idx
             Z[idx, :] = α_list[i] * x'
             i+=1
         else
             Z[idx, :] = c * ones(n)'
         end
     end
     Znoisy = Z + σ * randn(size(Z))
     shift_vector[sync_idx] = [50; 160; 200; 320; 420]
     Zshift_noisy = randshift(Znoisy, shift_vector)
    
    
     u, K = robust2shift_synced_vector(Zshift_noisy)
     u_a, K_a = robust2shiftscale_synced_vector(Zshift_noisy)

     top5_coord_idx = sort(sortperm(u[:]; rev=true)[1:5])
     top5_coord_idx_a = sort(sortperm(u_a[:];rev=true)[1:5])
     ## Plot disapleyd vector
     @show rank(K)
     @show rank(K_a)
     @show top5_coord_idx
     @show top5_coord_idx_a
     plot(
         joyplot(permutedims(Zshift_noisy); labels=labels),
         scatter(
             u[:];
             title="max_idx = $top5_coord_idx",
             xlabel="i",
             ylabel="U[i, 1]",
             ylim=(-0.1, 1)
     ),
     heatmap(K; aspect_ratio=1.0, axis=false, yflip=:true, grid=false),

     joyplot(permutedims(Zshift_noisy); labels=labels),
     scatter(
         u_a[:];
         title="max_idx = $top5_coord_idx_a",
         xlabel="i",
         ylabel="U[i, 1]",
         ylim=(-0.1, 1)
     ),
     heatmap(K_a; aspect_ratio=1.0, axis=false, yflip=:true, grid=false),
     layout = (2,3),
     size=(1200, 800))
end


# A substantially faster algorithm using the FFT

## Signal Processing 201: Discrete Fourier transform of circularly shifted waveforms

The Discrete Fourier transform of a vector is given by an unitary transformation $x \mapsto \mathcal{F}(x)$ given by

$$ \mathcal{F}(x) = Q x,$$

where $Q$ is the $n \times n$ [DFT matrix](https://en.wikipedia.org/wiki/DFT_matrix) defined as

$$Q = \begin{bmatrix} Q_{jk} \end{bmatrix} = \dfrac{1}{\sqrt{n}} \left[ \omega^{jk} \right]_{j,k = 1}^{n},$$

where $\omega = e^{-2\pi\,i/n}$.

**If the vector $y$ is related to $x$ via a circular shift** of $\tau$ so that 

$$ y = \textrm{shift}_{\tau}(x),$$
    
then via the [Shift Theorem](https://en.wikipedia.org/wiki/Discrete_Fourier_transform#Shift_theorem) it can be shown that
    
$$\mathcal{F}(y)_k = e^{-2\pi\, i \,k \tau/n} \mathcal{F}(x)_k.$$


Consequently 

$$|\mathcal{F}(y)_k| = |\mathcal{F}(x)_k|,$$

and so

$$|\mathcal{F}(y)| = |\mathcal{F}(x)|.$$



In [ ]:
using FFTW: fft
absYdata_fft = abs.(fft(Ydata,2))
@time u₁ = synced_vector_robust2mean(absYdata_fft)[:];

In [ ]:
top5_coord_idx = sort(sortperm(u₁; rev=true)[1:5])
scatter(u₁;
        title="max_idx = $top5_coord_idx", 
        xlabel="i", 
        ylabel="U{i,1}",
        ylim=(-0.1, 1))

This shows how we can develop faster algorithms with a deeper knowledge of the Fourier transform.

## Challenge problem 

**Exercise**: 

How would you utilize the DFT to speed up the `robust2shiftscale_synced_vector` algorithm. 

Hint: Express the normalization you performed for the `K` matrix in terms of the `absYdata_fft` matrix.

By DFT, we don't need to find max cross correlation, which can saves lots of time. 

In [ ]:
##TODO: Your code showing the speedup and that the algorithm works 
function robust2shiftscale_synced_vector_fft(A::Matrix, maxlag=size(A, 2))
    m, n = size(A)
    c = mean(A; dims = 2)
    A  = A .- c
    A_norm = vcat((norm(A[i,:]) for i in 1:size(A, 1))...)
    
    index = findall(A_norm .> 1e-9)
    A[index, :] = A[index, :] ./ A_norm[index]
    
    abs_A_fft = abs.(fft(A, 2))
    
    U = svds(abs_A_fft; nsv=1)[1].U
    U = U * sign.(U[findmax(abs.(U))[2]])
    return U, abs_A_fft
end
    
α_list = [3,6,9,12,15]
@manipulate for c in (0, 0.05, 0.1, 0.5), σ in (0, 0.01, 0.05,0.5)
     Z = zeros(m, n)
     i=1
    for idx in 1:m
         if idx in sync_idx
             Z[idx, :] = α_list[i] * x'
             i+=1
         else
             Z[idx, :] = c * ones(n)'
         end
     end
     Znoisy = Z + σ * randn(size(Z))
     shift_vector[sync_idx] = [50; 160; 200; 320; 420]
     Zshift_noisy = randshift(Znoisy, shift_vector)
    
     @time u_a, K_a = robust2shiftscale_synced_vector(Zshift_noisy)
     @time u, K = robust2shiftscale_synced_vector_fft(Zshift_noisy)

     top5_coord_idx = sort(sortperm(u[:]; rev=true)[1:5])
     top5_coord_idx_a = sort(sortperm(u_a[:];rev=true)[1:5])
     # Plot disapleyd vector
     @show rank(K)
     @show rank(K_a)
     @show top5_coord_idx
     @show top5_coord_idx_a
     plot(
         scatter(
             u[:];
             title="FFT max_idx = $top5_coord_idx",
             xlabel="i",
             ylabel="U[i, 1]",
             ylim=(-0.1, 1)
     ),

     scatter(
         u_a[:];
         title="original Algorithm max_idx = $top5_coord_idx_a\n",
         xlabel="i",
         ylabel="U[i, 1]",
         ylim=(-0.1, 1)
     ),
     layout = (1,2),
     size=(900, 400))
end
